In [ ]:
import warnings
warnings.filterwarnings("ignore")
import shutil
import os
import pandas as pd
import matplotlib
matplotlib.use(u'nbAgg')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import tqdm
# !pip install downcast
from downcast import reduce

In [ ]:
with open(r'/content/df_final1.pkl', 'rb') as f:
    df_final_= pickle.load(f)

## 1. Adding day feature to simple features.

In [ ]:
with open(r'/content/df_final1.pkl', 'rb') as f:
    df_final_= pickle.load(f)

In [ ]:
df_final=df_final_[0]

In [ ]:
data=df_final[['sold','is_high_sale_months',	'is_high_sale_day'	,'is_christmas','wm_yr_wk',	'wday',	'month',	'year',	'snap_CA',	'snap_TX',	'snap_WI',	'event_name_1_',	'event_type_1_',	'event_name_2_',	'event_type_2_'	,'id_'	,'item_id_',	'dept_id_',	'cat_id_',	'store_id_'	,'state_id_'	,'day']]

In [ ]:
data.shape

(59181090, 22)

* dividing data into input data and output data

In [ ]:
x=data[['is_high_sale_months',	'is_high_sale_day'	,'is_christmas','wm_yr_wk',	'wday',	'month',	'year',	'snap_CA',	'snap_TX',	'snap_WI',	'event_name_1_',	'event_type_1_',	'event_name_2_',	'event_type_2_'	,'id_'	,'item_id_',	'dept_id_',	'cat_id_',	'store_id_'	,'state_id_'	,'day']]

In [ ]:
y=data[['day','sold']]

In [ ]:
x.head()

,is_high_sale_months,is_high_sale_day,is_christmas,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14370,1437,3,1,0,0,1
1,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14380,1438,3,1,0,0,1
2,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14390,1439,3,1,0,0,1
3,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14400,1440,3,1,0,0,1
4,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14410,1441,3,1,0,0,1


In [ ]:
y.head()

,day,sold
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


* Dividing data into train and test

In [ ]:
x_train=x.loc[x['day']<1914]
x_test=x.loc[x['day']>=1914]

In [ ]:
# train=ma_df.loc[ma_df['day']<1914]
# test=ma_df.loc[ma_df['day']>=1914]
# print("train {}".format(train.shape))
# print("\ntest {}".format(test.shape))

In [ ]:
print("x_train {}".format(x_train.shape))
print("\nx_test {}".format(x_test.shape))

x_train (58327370, 21)

x_test (853720, 21)


In [ ]:
x_train.head()

,is_high_sale_months,is_high_sale_day,is_christmas,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14370,1437,3,1,0,0,1
1,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14380,1438,3,1,0,0,1
2,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14390,1439,3,1,0,0,1
3,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14400,1440,3,1,0,0,1
4,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14410,1441,3,1,0,0,1


In [ ]:
x_test.head()

,is_high_sale_months,is_high_sale_day,is_christmas,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
58327370,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14370,1437,3,1,0,0,1914
58327371,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14380,1438,3,1,0,0,1914
58327372,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14390,1439,3,1,0,0,1914
58327373,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14400,1440,3,1,0,0,1914
58327374,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14410,1441,3,1,0,0,1914


In [ ]:
y_train=y.loc[y['day']<1914]
y_test=y.loc[y['day']>=1914]

In [ ]:
print("y_train {}".format(y_train.shape))
print("\ny_test {}".format(y_test.shape))

y_train (58327370, 2)

y_test (853720, 2)


In [ ]:
y_train.drop(columns=['day'],inplace=True)
y_test.drop(columns=['day'],inplace=True)

In [ ]:
print("y_train {}".format(y_train.shape))
print("\ny_test {}".format(y_test.shape))

y_train (58327370, 1)

y_test (853720, 1)


In [ ]:
with open(r'spliting_data_day_feature.pkl','wb') as f:
    pickle.dump([x_train,x_test,y_train,y_test], f)

In [ ]:
with open(r'/content/spliting_data_day_feature.pkl', 'rb') as f:
    x_train,x_test,y_train,y_test= pickle.load(f)

### 2.1 Linear regression with lasso. (L1 regularization)

* model training

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(x_train,y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720,)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using Linear regreation with L1 :',RMSE)

RMSE using Linear regreation with L1 : 13.05074764699362


### 2.2 Linear regression with ridge. (L2 regularization)

In [ ]:
from sklearn import linear_model
clf = linear_model.Ridge(alpha=0.1)
clf.fit(x_train,y_train)

Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720, 1)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using Linear regreation with L2:',RMSE)

RMSE using Linear regreation with L2: 12.968768355698733


### 2.3 DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
clf=DecisionTreeRegressor(random_state=0)
clf.fit(x_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720,)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using DecisionTreeRegressor:',RMSE)

RMSE using DecisionTreeRegressor: 8.678277421168533


### 2.4 LGBMRegressor

In [ ]:
from lightgbm import LGBMRegressor
clf=LGBMRegressor(n_jobs=- 1)
clf.fit(x_train,y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720,)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using LGBMRegressor:',RMSE)

RMSE using LGBMRegressor: 11.698133629352517


## 3. Adding lag feature to simple features.

In [ ]:
with open(r'/content/drive/MyDrive/Project Files/df_final1.pkl', 'rb') as f:
    df_final = pickle.load(f)

In [ ]:
df_final[0]

,sold,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_mean_7,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,expanding_mean,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
0,0,11101,1,1,2011,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14370,1437,3,1,0,0,1
1,0,11101,1,1,2011,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14380,1438,3,1,0,0,1
2,0,11101,1,1,2011,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14390,1439,3,1,0,0,1
3,0,11101,1,1,2011,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14400,1440,3,1,0,0,1
4,0,11101,1,1,2011,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14410,1441,3,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,1,11617,2,5,2016,0,0,0,1,1,0,1.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.571289,0.643066,0.643066,0.571289,0.571289,0.535645,19,2,3,1,14329,1432,2,0,9,2,1941
59181086,0,11617,2,5,2016,0,0,0,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.285645,0.142822,0.285645,0.257080,0.261963,0.375000,19,2,3,1,14339,1433,2,0,9,2,1941
59181087,2,11617,2,5,2016,0,0,0,1,1,0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,1.0,0.856934,1.000000,0.785645,0.742676,0.785645,0.893555,19,2,3,1,14349,1434,2,0,9,2,1941
59181088,0,11617,2,5,2016,0,0,0,1,1,0,1.0,1.0,1.0,0.0,3.0,1.0,2.0,1.0,4.0,1.0,3.0,1.857422,1.428711,1.321289,1.286133,1.190430,0.380615,19,2,3,1,14359,1435,2,0,9,2,1941


In [ ]:
df_final=df_final[0]

In [ ]:
# ,'lag_1',	'lag_7',	'lag_14',	'lag_21',	'lag_28',	'lag_35',	'lag_42',	'lag_49',	'lag_56',	'lag_63',	'lag_70',

In [ ]:
data=df_final[['sold','lag_1',	'lag_7',	'lag_14',	'lag_21',	'lag_28',	'lag_35',	'lag_42',	'lag_49',	'lag_56',	'lag_63',	'lag_70','is_high_sale_months',	'is_high_sale_day'	,'is_christmas','wm_yr_wk',	'wday',	'month',	'year',	'snap_CA',	'snap_TX',	'snap_WI',	'event_name_1_',	'event_type_1_',	'event_name_2_',	'event_type_2_'	,'id_'	,'item_id_',	'dept_id_',	'cat_id_',	'store_id_'	,'state_id_'	,'day']]

In [ ]:
data.shape

(59181090, 33)

* dividing data into input data and output data

In [ ]:
x=data[['lag_1','lag_7','lag_14',	'lag_21',	'lag_28',	'lag_35',	'lag_42',	'lag_49',	'lag_56',	'lag_63',	'lag_70','is_high_sale_months',	'is_high_sale_day'	,'is_christmas','wm_yr_wk',	'wday',	'month',	'year',	'snap_CA',	'snap_TX',	'snap_WI',	'event_name_1_',	'event_type_1_',	'event_name_2_',	'event_type_2_'	,'id_'	,'item_id_',	'dept_id_',	'cat_id_',	'store_id_'	,'state_id_'	,'day']]

In [ ]:
y=data[['day','sold']]

In [ ]:
x.head()

,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,is_high_sale_months,is_high_sale_day,is_christmas,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14370,1437,3,1,0,0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14380,1438,3,1,0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14390,1439,3,1,0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14400,1440,3,1,0,0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14410,1441,3,1,0,0,1


In [ ]:
y.head()

,day,sold
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


* Dividing data into train and test

In [ ]:
x_train=x.loc[(x['day']>730)&(x['day']<1914)]
x_test=x.loc[x['day']>=1914]

In [ ]:
print("x_train {}".format(x_train.shape))
print("\nx_test {}".format(x_test.shape))

x_train (36069670, 32)

x_test (853720, 32)


In [ ]:
x_train.head()

,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,is_high_sale_months,is_high_sale_day,is_christmas,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
22257700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,11301,3,1,2013,0,0,0,19,2,3,1,14370,1437,3,1,0,0,731
22257701,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,2.0,0.0,0.0,0,0,0,11301,3,1,2013,0,0,0,19,2,3,1,14380,1438,3,1,0,0,731
22257702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,11301,3,1,2013,0,0,0,19,2,3,1,14390,1439,3,1,0,0,731
22257703,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,11301,3,1,2013,0,0,0,19,2,3,1,14400,1440,3,1,0,0,731
22257704,6.0,2.0,0.0,1.0,0.0,4.0,2.0,1.0,0.0,1.0,0.0,0,0,0,11301,3,1,2013,0,0,0,19,2,3,1,14410,1441,3,1,0,0,731


In [ ]:
x_test.head()

,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,is_high_sale_months,is_high_sale_day,is_christmas,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
58327370,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,0.0,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14370,1437,3,1,0,0,1914
58327371,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14380,1438,3,1,0,0,1914
58327372,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14390,1439,3,1,0,0,1914
58327373,2.0,4.0,2.0,1.0,0.0,4.0,3.0,3.0,0.0,2.0,0.0,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14400,1440,3,1,0,0,1914
58327374,4.0,0.0,2.0,0.0,1.0,2.0,2.0,2.0,1.0,0.0,2.0,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14410,1441,3,1,0,0,1914


In [ ]:
y_train=y.loc[(x['day']>730)&(x['day']<1914)]
y_test=y.loc[y['day']>=1914]

In [ ]:
print("y_train {}".format(y_train.shape))
print("\ny_test {}".format(y_test.shape))

y_train (36069670, 2)

y_test (853720, 2)


In [ ]:
y_train.drop(columns=['day'],inplace=True)
y_test.drop(columns=['day'],inplace=True)

In [ ]:
print("y_train {}".format(y_train.shape))
print("\ny_test {}".format(y_test.shape))

y_train (36069670, 1)

y_test (853720, 1)


In [ ]:
with open(r'spliting_data_lag_feature.pkl','wb') as f:
    pickle.dump([x_train,x_test,y_train,y_test], f)

In [ ]:
with open(r'spliting_data_lag_feature.pkl', 'rb') as f:
    x_train,x_test,y_train,y_test= pickle.load(f)

In [ ]:
# with open(r'/content/drive/MyDrive/Project Files/df_final1.pkl', 'rb') as f:
#     df_final = pickle.load(f)

### 3.1 Linear regression with lasso. (L1 regularization)

* model training

In [ ]:
x_train.head()

,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,is_high_sale_months,is_high_sale_day,is_christmas,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14370,1437,3,1,0,0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14380,1438,3,1,0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14390,1439,3,1,0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14400,1440,3,1,0,0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14410,1441,3,1,0,0,1


In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(x_train,y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720,)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using Linear regreation with L1 :',RMSE)

RMSE using Linear regreation with L1 : 4.3256965


### 3.2 Linear regression with ridge. (L2 regularization)

In [ ]:
x_train=reduce(x_train)
y_train=reduce(y_train)

In [ ]:
x_train.shape

(36069670, 32)

In [ ]:
from sklearn import linear_model
clf = linear_model.Ridge(alpha=0.1)
clf.fit(x_train,y_train)

Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720, 1)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using Linear regreation with L2:',RMSE)

RMSE using Linear regreation with L2: 4.304025


### 3.3 DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
clf=DecisionTreeRegressor(random_state=0)
clf.fit(x_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720,)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using DecisionTreeRegressor:',RMSE)

RMSE using DecisionTreeRegressor: 11.638208077589843


### 3.4 LGBMRegressor

In [ ]:
# with open(r'spliting_data_lag_feature.pkl', 'rb') as f:
#     x_train,x_test,y_train,y_test= pickle.load(f)

In [ ]:
from lightgbm import LGBMRegressor
clf=LGBMRegressor()
clf.fit(x_train,y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720,)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using LGBMRegressor:',RMSE)

RMSE using LGBMRegressor: 4.072276056683684


## 4 Adding rolling window features

In [ ]:
with open(r'/content/drive/MyDrive/Project Files/df_final1.pkl', 'rb') as f:
    df_final = pickle.load(f)

In [ ]:
df_final[0]

,sold,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_mean_7,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,expanding_mean,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
0,0,11101,1,1,2011,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14370,1437,3,1,0,0,1
1,0,11101,1,1,2011,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14380,1438,3,1,0,0,1
2,0,11101,1,1,2011,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14390,1439,3,1,0,0,1
3,0,11101,1,1,2011,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14400,1440,3,1,0,0,1
4,0,11101,1,1,2011,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14410,1441,3,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,1,11617,2,5,2016,0,0,0,1,1,0,1.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.571289,0.643066,0.643066,0.571289,0.571289,0.535645,19,2,3,1,14329,1432,2,0,9,2,1941
59181086,0,11617,2,5,2016,0,0,0,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.285645,0.142822,0.285645,0.257080,0.261963,0.375000,19,2,3,1,14339,1433,2,0,9,2,1941
59181087,2,11617,2,5,2016,0,0,0,1,1,0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,1.0,0.856934,1.000000,0.785645,0.742676,0.785645,0.893555,19,2,3,1,14349,1434,2,0,9,2,1941
59181088,0,11617,2,5,2016,0,0,0,1,1,0,1.0,1.0,1.0,0.0,3.0,1.0,2.0,1.0,4.0,1.0,3.0,1.857422,1.428711,1.321289,1.286133,1.190430,0.380615,19,2,3,1,14359,1435,2,0,9,2,1941


In [ ]:
df_final=df_final[0]

In [ ]:
# ,'lag_1',	'lag_7',	'lag_14',	'lag_21',	'lag_28',	'lag_35',	'lag_42',	'lag_49',	'lag_56',	'lag_63',	'lag_70',
# ,'rolling_mean_7',	'rolling_mean_14'	,'rolling_mean_28',	'rollingmean_35'	,'rolling_mean_42',

In [ ]:
data=df_final[['sold','rolling_mean_7',	'rolling_mean_14'	,'rolling_mean_28','rolling_mean_35','rolling_mean_42','lag_1',	'lag_7',	'lag_14',	'lag_21',	'lag_28',	'lag_35',	'lag_42',	'lag_49',	'lag_56',	'lag_63',	'lag_70','is_high_sale_months',	'is_high_sale_day'	,'is_christmas','wm_yr_wk',	'wday',	'month',	'year',	'snap_CA',	'snap_TX',	'snap_WI',	'event_name_1_',	'event_type_1_',	'event_name_2_',	'event_type_2_'	,'id_'	,'item_id_',	'dept_id_',	'cat_id_',	'store_id_'	,'state_id_'	,'day']]

In [ ]:
data.shape

(59181090, 38)

* dividing data into input data and output data

In [ ]:
x=data[['rolling_mean_7',	'rolling_mean_14'	,'rolling_mean_28',	'rolling_mean_35','rolling_mean_42','lag_1','lag_7','lag_14',	'lag_21',	'lag_28',	'lag_35',	'lag_42',	'lag_49',	'lag_56',	'lag_63',	'lag_70','is_high_sale_months',	'is_high_sale_day'	,'is_christmas','wm_yr_wk',	'wday',	'month',	'year',	'snap_CA',	'snap_TX',	'snap_WI',	'event_name_1_',	'event_type_1_',	'event_name_2_',	'event_type_2_'	,'id_'	,'item_id_',	'dept_id_',	'cat_id_',	'store_id_'	,'state_id_'	,'day']]

In [ ]:
y=data[['day','sold']]

In [ ]:
x.head()

,rolling_mean_7,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,is_high_sale_months,is_high_sale_day,is_christmas,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14370,1437,3,1,0,0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14380,1438,3,1,0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14390,1439,3,1,0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14400,1440,3,1,0,0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14410,1441,3,1,0,0,1


In [ ]:
y.head()

,day,sold
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


* Dividing data into train and test

In [ ]:
x_train=x.loc[(x['day']>900)&(x['day']<1914)]
x_test=x.loc[x['day']>=1914]

In [ ]:
print("x_train {}".format(x_train.shape))
print("\nx_test {}".format(x_test.shape))

x_train (30886370, 37)

x_test (853720, 37)


In [ ]:
x_train.head()

,rolling_mean_7,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,is_high_sale_months,is_high_sale_day,is_christmas,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
27441000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,11325,5,7,2013,0,0,0,19,2,3,1,14370,1437,3,1,0,0,901
27441001,0.428467,0.285645,0.285645,0.314209,0.309570,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0,0,0,11325,5,7,2013,0,0,0,19,2,3,1,14380,1438,3,1,0,0,901
27441002,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,11325,5,7,2013,0,0,0,19,2,3,1,14390,1439,3,1,0,0,901
27441003,0.000000,0.142822,0.643066,0.714355,0.714355,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,7.0,1.0,0,0,0,11325,5,7,2013,0,0,0,19,2,3,1,14400,1440,3,1,0,0,901
27441004,1.428711,1.500000,1.107422,1.057617,1.071289,0.0,3.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0,0,0,11325,5,7,2013,0,0,0,19,2,3,1,14410,1441,3,1,0,0,901


In [ ]:
x_test.head()

,rolling_mean_7,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,is_high_sale_months,is_high_sale_day,is_christmas,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
58327370,1.000000,0.928711,0.928711,0.971191,1.071289,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,0.0,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14370,1437,3,1,0,0,1914
58327371,0.142822,0.071411,0.035706,0.171387,0.142822,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14380,1438,3,1,0,0,1914
58327372,0.714355,1.000000,0.571289,0.514160,0.452393,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14390,1439,3,1,0,0,1914
58327373,2.000000,1.928711,1.821289,2.056641,2.072266,2.0,4.0,2.0,1.0,0.0,4.0,3.0,3.0,0.0,2.0,0.0,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14400,1440,3,1,0,0,1914
58327374,1.857422,1.357422,1.357422,1.228516,1.286133,4.0,0.0,2.0,0.0,1.0,2.0,2.0,2.0,1.0,0.0,2.0,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14410,1441,3,1,0,0,1914


In [ ]:
y_train=y.loc[(x['day']>900)&(x['day']<1914)]
y_test=y.loc[y['day']>=1914]

In [ ]:
print("y_train {}".format(y_train.shape))
print("\ny_test {}".format(y_test.shape))

y_train (30886370, 2)

y_test (853720, 2)


In [ ]:
y_train.drop(columns=['day'],inplace=True)
y_test.drop(columns=['day'],inplace=True)

In [ ]:
print("y_train {}".format(y_train.shape))
print("\ny_test {}".format(y_test.shape))

y_train (30886370, 1)

y_test (853720, 1)


In [ ]:
with open(r'spliting_data_rolling_window_feature_900.pkl','wb') as f:
    pickle.dump([x_train,x_test,y_train,y_test], f)

In [ ]:
with open(r'spliting_data_rolling_window_feature.pkl', 'rb') as f:
    x_train,x_test,y_train,y_test= pickle.load(f)

In [ ]:
# with open(r'/content/drive/MyDrive/Project Files/df_final1.pkl', 'rb') as f:
#     df_final = pickle.load(f)

### 4.1 Linear regression with lasso. (L1 regularization)

In [ ]:
x_train.head()

,rolling_mean_7,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,is_high_sale_months,is_high_sale_day,is_christmas,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
22257700,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,11301,3,1,2013,0,0,0,19,2,3,1,14370,1437,3,1,0,0,731
22257701,0.142822,0.214233,0.178589,0.142822,0.500000,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,2.0,0.0,0.0,0,0,0,11301,3,1,2013,0,0,0,19,2,3,1,14380,1438,3,1,0,0,731
22257702,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,11301,3,1,2013,0,0,0,19,2,3,1,14390,1439,3,1,0,0,731
22257703,1.000000,0.500000,0.250000,0.199951,0.166626,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,11301,3,1,2013,0,0,0,19,2,3,1,14400,1440,3,1,0,0,731
22257704,1.571289,1.286133,1.463867,1.485352,1.642578,6.0,2.0,0.0,1.0,0.0,4.0,2.0,1.0,0.0,1.0,0.0,0,0,0,11301,3,1,2013,0,0,0,19,2,3,1,14410,1441,3,1,0,0,731


In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(x_train,y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720,)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using Linear regreation with L1 :',RMSE)

RMSE using Linear regreation with L1 : 3.3268466


### 4.2 Linear regression with ridge. (L2 regularization)

In [ ]:
x_train=reduce(x_train)
y_train=reduce(y_train)

In [ ]:
x_train.shape

(36069670, 37)

In [ ]:
from sklearn import linear_model
clf = linear_model.Ridge(alpha=0.1)
clf.fit(x_train,y_train)

Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720, 1)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using Linear regreation with L2:',RMSE)

RMSE using Linear regreation with L2: 3.179003


### 4.3 DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
clf=DecisionTreeRegressor(random_state=0)
clf.fit(x_train,y_train)

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720,)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using DecisionTreeRegressor:',RMSE)

RMSE using DecisionTreeRegressor: 7.4056669634072065


### 4.4 LGBMRegressor

In [ ]:
with open(r'/content/spliting_data_rolling_window_feature_900.pkl', 'rb') as f:
    x_train,x_test,y_train,y_test= pickle.load(f)

In [ ]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30886370 entries, 27441000 to 58327369
Data columns (total 37 columns):
 #   Column               Dtype  
---  ------               -----  
 0   rolling_mean_7       float16
 1   rolling_mean_14      float16
 2   rolling_mean_28      float16
 3   rolling_mean_35      float16
 4   rolling_mean_42      float16
 5   lag_1                float16
 6   lag_7                float16
 7   lag_14               float16
 8   lag_21               float16
 9   lag_28               float16
 10  lag_35               float16
 11  lag_42               float16
 12  lag_49               float16
 13  lag_56               float16
 14  lag_63               float16
 15  lag_70               float16
 16  is_high_sale_months  int8   
 17  is_high_sale_day     int8   
 18  is_christmas         int8   
 19  wm_yr_wk             int16  
 20  wday                 int8   
 21  month                int8   
 22  year                 int16  
 23  snap_CA              in

In [ ]:
# x_train=reduce(x_train)
# y_train=reduce(y_train)

In [ ]:
# with open(r'spliting_data_lag_feature.pkl', 'rb') as f:
#     x_train,x_test,y_train,y_test= pickle.load(f)

In [ ]:
from lightgbm import LGBMRegressor
clf=LGBMRegressor()
clf.fit(x_train,y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720,)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using LGBMRegressor:',RMSE)

RMSE using LGBMRegressor: 3.046377735306875


## 5 Adding expanding window features

In [ ]:
# with open(r'/content/drive/MyDrive/Project Files/df_final1.pkl', 'rb') as f:
#     df_final = pickle.load(f)

In [ ]:
df_final[0]

,sold,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_mean_7,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,expanding_mean,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
0,0,11101,1,1,2011,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14370,1437,3,1,0,0,1
1,0,11101,1,1,2011,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14380,1438,3,1,0,0,1
2,0,11101,1,1,2011,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14390,1439,3,1,0,0,1
3,0,11101,1,1,2011,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14400,1440,3,1,0,0,1
4,0,11101,1,1,2011,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14410,1441,3,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,1,11617,2,5,2016,0,0,0,1,1,0,1.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.571289,0.643066,0.643066,0.571289,0.571289,0.535645,19,2,3,1,14329,1432,2,0,9,2,1941
59181086,0,11617,2,5,2016,0,0,0,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.285645,0.142822,0.285645,0.257080,0.261963,0.375000,19,2,3,1,14339,1433,2,0,9,2,1941
59181087,2,11617,2,5,2016,0,0,0,1,1,0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,1.0,0.856934,1.000000,0.785645,0.742676,0.785645,0.893555,19,2,3,1,14349,1434,2,0,9,2,1941
59181088,0,11617,2,5,2016,0,0,0,1,1,0,1.0,1.0,1.0,0.0,3.0,1.0,2.0,1.0,4.0,1.0,3.0,1.857422,1.428711,1.321289,1.286133,1.190430,0.380615,19,2,3,1,14359,1435,2,0,9,2,1941


In [ ]:
df_final=df_final[0]

In [ ]:
# ,'lag_1',	'lag_7',	'lag_14',	'lag_21',	'lag_28',	'lag_35',	'lag_42',	'lag_49',	'lag_56',	'lag_63',	'lag_70',
# ,'rolling_mean_7',	'rolling_mean_14'	,'rolling_mean_28',	'rollingmean_35'	,'rolling_mean_42',

In [ ]:
data=df_final[['sold','expanding_mean','rolling_mean_7',	'rolling_mean_14'	,'rolling_mean_28','rolling_mean_35','rolling_mean_42','lag_1',	'lag_7',	'lag_14',	'lag_21',	'lag_28',	'lag_35',	'lag_42',	'lag_49',	'lag_56',	'lag_63',	'lag_70','is_high_sale_months',	'is_high_sale_day'	,'is_christmas','wm_yr_wk',	'wday',	'month',	'year',	'snap_CA',	'snap_TX',	'snap_WI',	'event_name_1_',	'event_type_1_',	'event_name_2_',	'event_type_2_'	,'id_'	,'item_id_',	'dept_id_',	'cat_id_',	'store_id_'	,'state_id_'	,'day']]

In [ ]:
data.shape

(59181090, 39)

* dividing data into input data and output data

In [ ]:
x=data[['rolling_mean_7',	'expanding_mean','rolling_mean_14'	,'rolling_mean_28',	'rolling_mean_35','rolling_mean_42','lag_1','lag_7','lag_14',	'lag_21',	'lag_28',	'lag_35',	'lag_42',	'lag_49',	'lag_56',	'lag_63',	'lag_70','is_high_sale_months',	'is_high_sale_day'	,'is_christmas','wm_yr_wk',	'wday',	'month',	'year',	'snap_CA',	'snap_TX',	'snap_WI',	'event_name_1_',	'event_type_1_',	'event_name_2_',	'event_type_2_'	,'id_'	,'item_id_',	'dept_id_',	'cat_id_',	'store_id_'	,'state_id_'	,'day']]

In [ ]:
y=data[['day','sold']]

In [ ]:
x.head()

,rolling_mean_7,expanding_mean,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,is_high_sale_months,is_high_sale_day,is_christmas,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14370,1437,3,1,0,0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14380,1438,3,1,0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14390,1439,3,1,0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14400,1440,3,1,0,0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,11101,1,1,2011,0,0,0,19,2,3,1,14410,1441,3,1,0,0,1


In [ ]:
y.head()

,day,sold
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


* Dividing data into train and test

In [ ]:
x_train=x.loc[(x['day']>730)&(x['day']<1914)]
x_test=x.loc[x['day']>=1914]

In [ ]:
print("x_train {}".format(x_train.shape))
print("\nx_test {}".format(x_test.shape))

x_train (36069670, 38)

x_test (853720, 38)


In [ ]:
x_train.head()

,rolling_mean_7,expanding_mean,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,is_high_sale_months,is_high_sale_day,is_christmas,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
22257700,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,11301,3,1,2013,0,0,0,19,2,3,1,14370,1437,3,1,0,0,731
22257701,0.142822,0.210693,0.214233,0.178589,0.142822,0.500000,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,2.0,0.0,0.0,0,0,0,11301,3,1,2013,0,0,0,19,2,3,1,14380,1438,3,1,0,0,731
22257702,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,11301,3,1,2013,0,0,0,19,2,3,1,14390,1439,3,1,0,0,731
22257703,1.000000,1.165039,0.500000,0.250000,0.199951,0.166626,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,11301,3,1,2013,0,0,0,19,2,3,1,14400,1440,3,1,0,0,731
22257704,1.571289,0.848145,1.286133,1.463867,1.485352,1.642578,6.0,2.0,0.0,1.0,0.0,4.0,2.0,1.0,0.0,1.0,0.0,0,0,0,11301,3,1,2013,0,0,0,19,2,3,1,14410,1441,3,1,0,0,731


In [ ]:
x_test.head()

,rolling_mean_7,expanding_mean,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,is_high_sale_months,is_high_sale_day,is_christmas,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
58327370,1.000000,0.313477,0.928711,0.928711,0.971191,1.071289,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,0.0,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14370,1437,3,1,0,0,1914
58327371,0.142822,0.257568,0.071411,0.035706,0.171387,0.142822,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14380,1438,3,1,0,0,1914
58327372,0.714355,0.150513,1.000000,0.571289,0.514160,0.452393,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14390,1439,3,1,0,0,1914
58327373,2.000000,1.717773,1.928711,1.821289,2.056641,2.072266,2.0,4.0,2.0,1.0,0.0,4.0,3.0,3.0,0.0,2.0,0.0,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14400,1440,3,1,0,0,1914
58327374,1.857422,0.966797,1.357422,1.357422,1.228516,1.286133,4.0,0.0,2.0,0.0,1.0,2.0,2.0,2.0,1.0,0.0,2.0,1,0,0,11613,3,4,2016,0,0,0,19,2,3,1,14410,1441,3,1,0,0,1914


In [ ]:
y_train=y.loc[(x['day']>730)&(x['day']<1914)]
y_test=y.loc[y['day']>=1914]

In [ ]:
print("y_train {}".format(y_train.shape))
print("\ny_test {}".format(y_test.shape))

y_train (36069670, 2)

y_test (853720, 2)


In [ ]:
y_train.drop(columns=['day'],inplace=True)
y_test.drop(columns=['day'],inplace=True)

In [ ]:
print("y_train {}".format(y_train.shape))
print("\ny_test {}".format(y_test.shape))

y_train (30886370, 1)

y_test (853720, 1)


In [ ]:
with open(r'spliting_data_expanding_window_feature_730.pkl','wb') as f:
    pickle.dump([x_train,x_test,y_train,y_test], f)

In [ ]:
with open(r'spliting_data_expanding_window_feature_900.pkl', 'rb') as f:
    x_train,x_test,y_train,y_test= pickle.load(f)

In [ ]:
# with open(r'/content/drive/MyDrive/Project Files/df_final1.pkl', 'rb') as f:
#     df_final = pickle.load(f)

### 5.1 Linear regression with lasso. (L1 regularization)

* model training

In [ ]:
x_train.head()

,rolling_mean_7,expanding_mean,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,is_high_sale_months,is_high_sale_day,is_christmas,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
27441000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,11325,5,7,2013,0,0,0,19,2,3,1,14370,1437,3,1,0,0,901
27441001,0.428467,0.221924,0.285645,0.285645,0.314209,0.309570,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0,0,0,11325,5,7,2013,0,0,0,19,2,3,1,14380,1438,3,1,0,0,901
27441002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,11325,5,7,2013,0,0,0,19,2,3,1,14390,1439,3,1,0,0,901
27441003,0.000000,1.208984,0.142822,0.643066,0.714355,0.714355,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,7.0,1.0,0,0,0,11325,5,7,2013,0,0,0,19,2,3,1,14400,1440,3,1,0,0,901
27441004,1.428711,0.874512,1.500000,1.107422,1.057617,1.071289,0.0,3.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0,0,0,11325,5,7,2013,0,0,0,19,2,3,1,14410,1441,3,1,0,0,901


In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(x_train,y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720,)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using Linear regreation with L1 :',RMSE)

RMSE using Linear regreation with L1 : 3.324336


### 5.2 Linear regression with ridge. (L2 regularization)

In [ ]:
x_train=reduce(x_train)
y_train=reduce(y_train)

In [ ]:
x_train.shape

(30886370, 38)

In [ ]:
from sklearn import linear_model
clf = linear_model.Ridge(alpha=0.1)
clf.fit(x_train,y_train)

Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720, 1)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using Linear regreation with L2:',RMSE)

RMSE using Linear regreation with L2: 3.1794188


### 5.3 DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
clf=DecisionTreeRegressor(random_state=0)
clf.fit(x_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720,)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using DecisionTreeRegressor:',RMSE)

RMSE using DecisionTreeRegressor: 7.400911305814553


### 5.4 LGBMRegressor

In [ ]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30886370 entries, 27441000 to 58327369
Data columns (total 38 columns):
 #   Column               Dtype  
---  ------               -----  
 0   rolling_mean_7       float16
 1   expanding_mean       float16
 2   rolling_mean_14      float16
 3   rolling_mean_28      float16
 4   rolling_mean_35      float16
 5   rolling_mean_42      float16
 6   lag_1                float16
 7   lag_7                float16
 8   lag_14               float16
 9   lag_21               float16
 10  lag_28               float16
 11  lag_35               float16
 12  lag_42               float16
 13  lag_49               float16
 14  lag_56               float16
 15  lag_63               float16
 16  lag_70               float16
 17  is_high_sale_months  int8   
 18  is_high_sale_day     int8   
 19  is_christmas         int8   
 20  wm_yr_wk             int16  
 21  wday                 int8   
 22  month                int8   
 23  year                 in

In [ ]:
x_train=reduce(x_train)
y_train=reduce(y_train)

In [ ]:
# with open(r'spliting_data_lag_feature.pkl', 'rb') as f:
#     x_train,x_test,y_train,y_test= pickle.load(f)

In [ ]:
from lightgbm import LGBMRegressor
clf=LGBMRegressor()
clf.fit(x_train,y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
y_predicted=clf.predict(x_test)

In [ ]:
y_predicted.shape

(853720,)

In [ ]:
y_test['sold'].shape

(853720,)

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using LGBMRegressor:',RMSE)

RMSE using LGBMRegressor: 3.045406228352032


In [ ]:
from prettytable import PrettyTable
  
# Specify the Column Names while initializing the Table
myTable = PrettyTable(["Feature INFO ", "Model", "RMSE SCORE"])
  
# Add rows

myTable.add_row(["1. Adding Date based Features ", "    ", "    "])
myTable.add_row(["1.1 ", "Linear regression with L1 regularization", "13.05074764699362"])
myTable.add_row(["1.2","Linear regression with L2 regularization", "12.968768355698733"])
myTable.add_row(["1.3", "Decision Tree", " 8.678277421168533"])
myTable.add_row(["1.4", "LGBM Regressor", "11.698133629352517"])
myTable.add_row(["-------", "-------", "-------"])

myTable.add_row(["2. Adding Lag Features ", "    ", "    "])
myTable.add_row(["2.1 ", "Linear regression with L1 regularization", "4.3256965"])
myTable.add_row(["2.2","Linear regression with L2 regularization", "4.304025"])
myTable.add_row(["2.3", "Decision Tree", "11.638208077589843"])
myTable.add_row(["2.4", "LGBM Regressor", " 4.072276056683684"])
myTable.add_row(["-------", "-------", "-------"])

myTable.add_row(["3. Adding rolling window Features", "    ", "    "])
myTable.add_row(["3.1 ", "Linear regression with L1 regularization", "3.3268466"])
myTable.add_row(["3.2","Linear regression with L2 regularization", "3.179003"])
myTable.add_row(["3.3", "Decision Tree", "7.4056669634072065"])
myTable.add_row(["3.4", "LGBM Regressor", "3.046377735306875"])
myTable.add_row(["-------", "-------", "-------"])

myTable.add_row(["4. Adding Expanding window Features", "    ", "    "])
myTable.add_row(["4.1 ", "Linear regression with L1 regularization", "3.324336"])
myTable.add_row(["4.2","Linear regression with L2 regularization", "3.1794188"])
myTable.add_row(["4.3", "Decision Tree", "7.400911305814553"])
myTable.add_row(["4.4", "LGBM Regressor", "3.045406228352032"])
myTable.add_row(["-------", "-------", "-------"])

print(myTable)

+-------------------------------------+------------------------------------------+--------------------+
|            Feature INFO             |                  Model                   |     RMSE SCORE     |
+-------------------------------------+------------------------------------------+--------------------+
|    1. Adding Date based Features    |                                          |                    |
|                 1.1                 | Linear regression with L1 regularization | 13.05074764699362  |
|                 1.2                 | Linear regression with L2 regularization | 12.968768355698733 |
|                 1.3                 |              Decision Tree               |  8.678277421168533 |
|                 1.4                 |              LGBM Regressor              | 11.698133629352517 |
|               -------               |                 -------                  |      -------       |
|       2. Adding Lag Features        |                         